# Análisis del riesgo de incumplimiento de los prestatarios

Tu proyecto consiste en preparar un informe para la división de préstamos de un banco. Deberás averiguar si el estado civil y el número de hijos de un cliente tienen un impacto en el incumplimiento de pago de un préstamo. El banco ya tiene algunos datos sobre la solvencia crediticia de los clientes.

Tu informe se tendrá en cuenta al crear una **puntuación de crédito** para un cliente potencial. La **puntuación de crédito** se utiliza para evaluar la capacidad de un prestatario potencial para pagar su préstamo.

Para eso se estudiará la base de datos para corregir datos incongruentes y rellenar datos faltantes. Luego se estudiarán algunas variables para ver si tienen correlación con el incumplimiento de pago de un préstamo.

**Tabla de contenido**

1. Apertura de archivo y visualización de información general
2. Exploración de los datos
3. Transformación de los datos

    3.1 Restauración de valores ausentes de "total_income"
    
    3.2 Restauración de valores ausentes de "days_employed"
    
    
4. Clasificación de los datos
5. Comprobación de la hipótesis


## Abre el archivo de datos y mira la información general. 

In [4]:
#manipulación de la información
import pandas as pd
import random

In [5]:
#importando el DataFrame

try:
    bank_data = pd.read_csv("/datasets/credit_scoring_eng.csv")
except:
    bank_data = pd.read_csv("credit_scoring_eng.csv")

In [6]:
#observando la información general
bank_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


**Primeras observaciones**

- Se puede observar que hay 21525 filas y 12 columnas
- Las columnas "days_employed" y "total_income" tienen valores ausentes
- El tipo de cada dato parece estar correcto

## Ejercicio 1. Exploración de datos

**Descripción de los datos**
- `children` - el número de hijos en la familia
- `days_employed` - experiencia laboral en días
- `dob_years` - la edad del cliente en años
- `education` - la educación del cliente
- `education_id` - identificador de educación
- `family_status` - estado civil
- `family_status_id` - identificador de estado civil
- `gender` - género del cliente
- `income_type` - tipo de empleo
- `debt` - ¿había alguna deuda en el pago de un préstamo?
- `total_income` - ingreso mensual
- `purpose` - el propósito de obtener un préstamo

In [7]:
#mostrando las primeras 15 filas para identificar posibles errores
bank_data.head(15)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house
1,1,-4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase
2,0,-5623.422610,33,Secondary Education,1,married,0,M,employee,0,23341.752,purchase of the house
3,3,-4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding
5,0,-926.185831,27,bachelor's degree,0,civil partnership,1,M,business,0,40922.170,purchase of the house
6,0,-2879.202052,43,bachelor's degree,0,married,0,F,business,0,38484.156,housing transactions
7,0,-152.779569,50,SECONDARY EDUCATION,1,married,0,M,employee,0,21731.829,education
8,2,-6929.865299,35,BACHELOR'S DEGREE,0,civil partnership,1,F,employee,0,15337.093,having a wedding
9,0,-2188.756445,41,secondary education,1,married,0,M,employee,0,23108.150,purchase of the house for my family


- "days_employed" tiene valores negativos, lo cual no corresponde, ya que debería indicar el tiempo trabajado en días
- "education" y "purpose" tiene datos duplicados que están escritos de distinta manera, por lo que hay que arreglarlos para el mejor manejo de los datos
- "days_employed" y "total_income" tienen valores ausentes

In [8]:
#llamando a la tabla filtrada de las filas donde faltan datos en "days_employed"
nan_data = bank_data[bank_data["days_employed"].isnull() == True].reset_index(drop = True)
nan_data

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,0,NaN,65,secondary education,1,civil partnership,1,M,retiree,0,NaN,to have a wedding
1,0,NaN,41,secondary education,1,married,0,M,civil servant,0,NaN,education
2,0,NaN,63,secondary education,1,unmarried,4,F,retiree,0,NaN,building a real estate
3,0,NaN,50,secondary education,1,married,0,F,civil servant,0,NaN,second-hand car purchase
4,0,NaN,54,secondary education,1,civil partnership,1,F,retiree,1,NaN,to have a wedding
...,...,...,...,...,...,...,...,...,...,...,...,...
2169,2,NaN,47,Secondary Education,1,married,0,M,business,0,NaN,purchase of a car
2170,1,NaN,50,secondary education,1,civil partnership,1,F,employee,0,NaN,wedding ceremony
2171,0,NaN,48,BACHELOR'S DEGREE,0,married,0,F,business,0,NaN,building a property
2172,1,NaN,42,secondary education,1,married,0,F,employee,0,NaN,building a real estate


In [9]:
print(nan_data["days_employed"].isna().count())
print(nan_data["total_income"].isna().count())

2174
2174


- Al parecer los datos ausentes de la columna "days_employed" coincide con los datos ausentes de la columna "total_income"
- Esto podría tener sentido, ya que al no tener días trabajados no existe un ingreso económico

In [10]:
#filtrando la tabla en donde las dos columnas tengan valores ausentes
nan_data = bank_data[(bank_data["days_employed"].isna())&(bank_data["total_income"].isna())]
nan_data

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,secondary education,1,civil partnership,1,M,retiree,0,NaN,to have a wedding
26,0,NaN,41,secondary education,1,married,0,M,civil servant,0,NaN,education
29,0,NaN,63,secondary education,1,unmarried,4,F,retiree,0,NaN,building a real estate
41,0,NaN,50,secondary education,1,married,0,F,civil servant,0,NaN,second-hand car purchase
55,0,NaN,54,secondary education,1,civil partnership,1,F,retiree,1,NaN,to have a wedding
...,...,...,...,...,...,...,...,...,...,...,...,...
21489,2,NaN,47,Secondary Education,1,married,0,M,business,0,NaN,purchase of a car
21495,1,NaN,50,secondary education,1,civil partnership,1,F,employee,0,NaN,wedding ceremony
21497,0,NaN,48,BACHELOR'S DEGREE,0,married,0,F,business,0,NaN,building a property
21502,1,NaN,42,secondary education,1,married,0,F,employee,0,NaN,building a real estate


La tabla filtrada tiene 2174 filas que coincide con los 2174 datos ausentes de ambas columnas, por lo que se confirma que los datos faltantes son los mismos en ambas columnas "days_employed" y "total_income"


In [11]:
#calculando la proporción entre valores ausentes y el total de datos
quant_nan_data = nan_data["total_income"].isna().count()
total_data = 21525

"La proporción entre valores ausentes y el total de datos es: {:.0%}".format(quant_nan_data/total_data)

'La proporción entre valores ausentes y el total de datos es: 10%'

**Conclusión intermedia**

El número de filas de la tabla filtrada coincide con la cantidad de datos ausentes en la columna "days_employed" y en "total_income". Por lo que se podría concluir que al no tener días trabajados tampoco se tiene ingreso.

La proporción entre los valores ausentes y el total de datos es 10%, este es un valor considerable por lo que habrá que rellenar los valores ausentes.

A continuación se va a verificar si los datos ausentes podrían tener algo que ver con alguna característica del cliente. Para verificarlo, quita el símbolo "#" de la columna donde deseas revisar la frecuencia de sus datos.

In [12]:
#comprobando la distribución de los datos en la tabla de datos faltantes
for col in nan_data:
        print(nan_data[col].value_counts())
        print()

 0     1439
 1      475
 2      204
 3       36
 20       9
 4        7
-1        3
 5        1
Name: children, dtype: int64

Series([], Name: days_employed, dtype: int64)

34    69
40    66
42    65
31    65
35    64
36    63
47    59
41    59
30    58
28    57
58    56
57    56
54    55
56    54
38    54
52    53
37    53
33    51
39    51
50    51
43    50
45    50
49    50
51    50
29    50
46    48
55    48
48    46
44    44
53    44
60    39
62    38
61    38
32    37
64    37
23    36
27    36
26    35
59    34
63    29
25    23
24    21
65    20
66    20
21    18
22    17
67    16
0     10
68     9
71     5
69     5
20     5
70     3
72     2
19     1
73     1
Name: dob_years, dtype: int64

secondary education    1408
bachelor's degree       496
SECONDARY EDUCATION      67
Secondary Education      65
some college             55
Bachelor's Degree        25
BACHELOR'S DEGREE        23
primary education        19
Some College              7
SOME COLLEGE              7
Primary Educ

**Frecuencia de los datos**

- Más de la mitad de los clientes no tiene hijos, y hay algunos valores que no tienen sentido.
- Al parecer, la edad tampoco es una característica relevante, hay de todas las edades pero se concentra entre los 30 a los 50.
- Más de la mitad de los clientes tienen de nivel educativo la secundaria, pero hay otros tipos.
- Más de la mitad de la muestra está casado/a, pero hay tambien otros tipos.
- Hay más del doble de mujeres que hombres.
- Más de la mitad son empleados, luego tienen su negocio o son retirados o funcionarios.
- Casi toda la muestra nunca ha tenido deuda de pago.
- Los propósitos del crédito son comprar una casa, arreglar la casa, boda, comprar auto...


**Posibles razones por las que hay valores ausentes en los datos**

Por ahora no es posible observar ningún patrón, es probable que los datos ausentes sean al azar.

Se procede a comparar la tabla de datos faltantes con la original, a ver si la distribución de los datos varía mucho una con respecto a la otra. 

In [13]:
#comprobando la distribución de los datos en la tabla original

for col in bank_data:
        print(bank_data[col].value_counts())
        print()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

-8437.673028      1
-3507.818775      1
 354500.415854    1
-769.717438       1
-3963.590317      1
                 ..
-1099.957609      1
-209.984794       1
 398099.392433    1
-1271.038880      1
-1984.507589      1
Name: days_employed, Length: 19351, dtype: int64

35    617
40    609
41    607
34    603
38    598
42    597
33    581
39    573
31    560
36    555
44    547
29    545
30    540
48    538
37    537
50    514
43    513
32    510
49    508
28    503
45    497
27    493
56    487
52    484
47    480
54    479
46    475
58    461
57    460
53    459
51    448
59    444
55    443
26    408
60    377
25    357
61    355
62    352
63    269
64    265
24    264
23    254
65    194
22    183
66    183
67    167
21    111
0     101
68     99
69     85
70     65
71     58
20     51
72     33
19     14
73      8
74      6
75      1
Name: dob_years,

**Conclusión intermedia**

- En ambas tablas la distribución de la cantidad de hijos es la misma
- La distribución de la edad es muy similar
- La distribución del nivel educacional es el mismo
- El estado civil también tiene la misma distribución
- El sexo femenino sigue doblando al masculino
- El tipo de ingreso tiene la misma distribución también
- Si han sido deudores o no también
- Los propósitos no varían mucho

La distribución en el conjunto de datos original es muy similar a la de la tabla de valores ausentes, eso significa que no podemos sacar una conclusión a cerca de que es lo que está influyendo para que hayan datos ausentes.

Ahora se estudiara el comportamiento de la tabla eliminando los datos ausentes.

In [14]:
#creando una tabla sin datos faltantes
bank_data_filtered = bank_data.dropna(subset = ["days_employed", "total_income"]).reset_index(drop = True)
bank_data_filtered

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house
1,1,-4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase
2,0,-5623.422610,33,Secondary Education,1,married,0,M,employee,0,23341.752,purchase of the house
3,3,-4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding
...,...,...,...,...,...,...,...,...,...,...,...,...
19346,1,-4529.316663,43,secondary education,1,civil partnership,1,F,business,0,35966.698,housing transactions
19347,0,343937.404131,67,secondary education,1,married,0,F,retiree,0,24959.969,purchase of a car
19348,1,-2113.346888,38,secondary education,1,civil partnership,1,M,employee,1,14347.610,property
19349,3,-3112.481705,38,secondary education,1,married,0,M,employee,1,39054.888,buying my own car


In [15]:
#comprobando la distribución de los datos en la tabla sin datos faltantes

for col in bank_data_filtered:
        print(bank_data_filtered[col].value_counts())
        print()

 0     12710
 1      4343
 2      1851
 3       294
 20       67
-1        44
 4        34
 5         8
Name: children, dtype: int64

-8437.673028      1
-3507.818775      1
 354500.415854    1
-769.717438       1
-3963.590317      1
                 ..
-1099.957609      1
-209.984794       1
 398099.392433    1
-1271.038880      1
-1984.507589      1
Name: days_employed, Length: 19351, dtype: int64

35    553
41    548
38    544
40    543
34    534
42    532
33    530
39    522
44    503
29    495
31    495
48    492
36    492
37    484
30    482
32    473
43    463
50    463
49    458
27    457
45    447
28    446
56    433
52    431
46    427
54    424
47    421
53    415
59    410
58    405
57    404
51    398
55    395
26    373
60    338
25    334
61    317
62    314
24    243
63    240
64    228
23    218
65    174
22    166
66    163
67    151
21     93
0      91
68     90
69     80
70     62
71     53
20     46
72     31
19     13
73      7
74      6
75      1
Name: dob_years,

**Conclusiones**

Se puede concluir desde el resultado anterior, que las distribuciones de los datos de cada columna, ya sea en la tabla original, en la tabla de datos ausentes y en la tabla filtrada sin datos ausentes, son las mismas, por lo que los datos ausentes podrían ser al azar.

Esto se puede concluir ya que las filas de los datos ausentes no siguen ningún patrón, pues la distribución de las 3 tablas se mantiene solo por el hecho de que los datos con valores ausentes son al azar. 

Teniendo en cuenta lo anterior, si los datos faltantes fueran completamente aleatorios, no habría problema con eliminarlos, pero para saber si los datos son completamente aleatorios hace falta hacer un estudio más profundo, por lo que es mejor rellenar los datos faltantes con la media o mediana.

## Transformación de datos

Ahora se trabajará en la eliminación de duplicados y correción de la información, columna por columna.

**Primero se revisará la columna "education"**

In [16]:
#verificando la columna de educación
bank_data["education"].unique()

array(["bachelor's degree", 'secondary education', 'Secondary Education',
       'SECONDARY EDUCATION', "BACHELOR'S DEGREE", 'some college',
       'primary education', "Bachelor's Degree", 'SOME COLLEGE',
       'Some College', 'PRIMARY EDUCATION', 'Primary Education',
       'Graduate Degree', 'GRADUATE DEGREE', 'graduate degree'],
      dtype=object)

In [17]:
#arreglando los registros
bank_data["education"] = bank_data["education"].str.lower()

In [18]:
#comprobando los valores únicos de la columna
bank_data["education"].unique()

array(["bachelor's degree", 'secondary education', 'some college',
       'primary education', 'graduate degree'], dtype=object)

**Ahora se revisará la columna "children"**

In [19]:
#verificando la distribución de los valores en la columna "children"
bank_data["children"].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

Se puede observar que hay dos datos problemáticos, el -1 y el 20. Nadie puede tener -1 hijos y es muy poco probable que 76 personas de la muestra tengan 20 hijos en total. 
Se procede a ver la proporción de los datos erroneos con respecto al total.

In [20]:
total_data = 21525
wrong_data = 76+47
"La proporción entre datos problemáticos y el total de datos es: {:.0%}".format(wrong_data/total_data)

'La proporción entre datos problemáticos y el total de datos es: 1%'

- El -1 es posible que sea 1 y hubo algún error de tipeo.
- Es posible que el 20 sea 2 y hubo algún error de tipeo.

In [21]:
#arreglando los datos problemáticos
bank_data["children"] = bank_data["children"].replace(-1, 1)
bank_data["children"] = bank_data["children"].replace(20, 2)

In [22]:
#comprobando los datos de la columna children
bank_data["children"].unique()

array([1, 0, 3, 2, 4, 5], dtype=int64)

**Ahora se revisará la columna "days_employed"**

In [23]:
#verificando datos problemáticos en 'days_employed'
bank_data["days_employed"]

0         -8437.673028
1         -4024.803754
2         -5623.422610
3         -4124.747207
4        340266.072047
             ...      
21520     -4529.316663
21521    343937.404131
21522     -2113.346888
21523     -3112.481705
21524     -1984.507589
Name: days_employed, Length: 21525, dtype: float64

In [24]:
positive_days = bank_data[bank_data["days_employed"] > 0]["days_employed"]
print(positive_days.min())
print(positive_days.max())

328728.72060451825
401755.40047533


Los valores positivos son muy altos, se pasará a años, considerando un año como 365 días.

In [25]:
positive_days = bank_data[bank_data["days_employed"] > 0]["days_employed"]
print(positive_days.min()/365)
print(positive_days.max()/365)

900.6266317932007
1100.6997273296713


Es imposible que una persona pueda trabajar entre 900 y 1100 años. Este error en la base de datos puede deberse a un error en el ingreso de fechas de inicio y fin de etapa laboral.

In [26]:
#abordando los valores problemáticos
negative_days = bank_data[bank_data["days_employed"] < 0]["days_employed"].count()
total_days = bank_data["days_employed"].count()
"La proporción entre datos negativos y el total de datos es: {:.0%}".format(negative_days/total_days)

'La proporción entre datos negativos y el total de datos es: 82%'

La cantidad de datos negativos es muy alta. Podemos asumir que hubo un error de tipeo, al preguntar la fecha de ingreso a trabajar con la fecha de término se restaron al revés por lo que quedó la cantidad de días trabajados en negativo. 

Para resolverlo, se aplicara valor absoluto y se dejará la columna en años para poder observar mejor los datos.

In [27]:
#observando el valor mínimo y el máximo en días
print(bank_data["days_employed"].min())
print(bank_data["days_employed"].max())

-18388.949900568383
401755.40047533


In [28]:
#aplicando valor absoluto a la columna "days_employed"
bank_data["days_employed"] = bank_data["days_employed"].abs()

In [29]:
#creando una columna para guardar el tiempo trabajado en años
bank_data["years_employed"] = (bank_data["days_employed"])/365

In [30]:
#observando el valor mínimo y el máximo
print(bank_data["years_employed"].min())
print(bank_data["years_employed"].max())

0.06614146093282515
1100.6997273296713


In [31]:
bank_data[bank_data["years_employed"] > 100]["days_employed"].count()

3445

Es imposible que una persona trabaje más de 100 años. Se comprobó que sobre 100 años trabajados existen 3445 filas. Esto quizás se deba a que la persona ingreso mal su año de ingreso o de término.

In [32]:
print(bank_data[bank_data["years_employed"] > 100]["income_type"].value_counts())

retiree       3443
unemployed       2
Name: income_type, dtype: int64


Se puede observar que de todos los valores problemáticos, casi el 100% son personas que ya se encuentran retiradas.

In [33]:
#creando una tabla sin datos faltantes
bank_data_filtered = bank_data.dropna(subset = ["days_employed", "total_income"]).reset_index(drop = True)
bank_data_filtered

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,years_employed
0,1,8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house,23.116912
1,1,4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase,11.026860
2,0,5623.422610,33,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house,15.406637
3,3,4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education,11.300677
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding,932.235814
...,...,...,...,...,...,...,...,...,...,...,...,...,...
19346,1,4529.316663,43,secondary education,1,civil partnership,1,F,business,0,35966.698,housing transactions,12.409087
19347,0,343937.404131,67,secondary education,1,married,0,F,retiree,0,24959.969,purchase of a car,942.294258
19348,1,2113.346888,38,secondary education,1,civil partnership,1,M,employee,1,14347.610,property,5.789991
19349,3,3112.481705,38,secondary education,1,married,0,M,employee,1,39054.888,buying my own car,8.527347


In [34]:
bank_data_filtered[(bank_data_filtered["income_type"] == "unemployed") & (bank_data_filtered["years_employed"] < 100)]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,years_employed


In [35]:
bank_data_filtered[(bank_data_filtered["income_type"] == "retiree") & (bank_data_filtered["years_employed"] < 100)]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,years_employed


Todas las personas con "income_type" = unemployed o retiree tienen más de 100 años trabajados, por lo que la columna "years_employed" no se puede arreglar en base a "income_type".

In [36]:
#revisando los años de las personas con años trabajados mayor a 100
bank_data_filtered[bank_data_filtered["years_employed"] > 100]["dob_years"].unique()

array([53, 57, 67, 62, 68, 63, 64, 61, 54,  0, 65, 59, 60, 71, 38, 58, 55,
       73, 52, 66, 56, 44, 69, 51, 49, 46, 50, 41, 48, 42, 72, 22, 37, 70,
       39, 45, 47, 74, 43, 31, 34, 33, 36, 32, 40, 35, 27, 26, 28],
      dtype=int64)

In [37]:
#arreglando la columna de días trabajados que corresponden a más de 100 años
bank_data.loc[(bank_data["years_employed"] > 100)&(bank_data["dob_years"] > 0), "days_employed"] = (random.uniform(1,(bank_data["dob_years"]-15)))*365

Como las personas con más de 100 años trabajados eran todos los retirados y desempleados, se determinó que sus años de trabajo serían 60, si bien es un número al azar que trabajaría una persona retirada, esto no va a afectar en el análisis de las hipotesis planteadas en el proyecto, solo servirá para rellenar los datos faltantes.

In [38]:
#comprobando que se arreglaron los días trabajados que tenían valor mayor a 900 años
print(bank_data["days_employed"].min())
print(bank_data["days_employed"].max())

24.14163324048118
400992.3757037226


In [39]:
#arreglando la columna de años trabajados
bank_data["years_employed"] = (bank_data["days_employed"])/365

In [40]:
#revisando si dió resultado
print(bank_data["years_employed"].max())

1098.6092485033496


Queda solo un dato, que es el que equivale a "dob_years" = 0, por lo que primero se arreglará ese valor y luego se le asignará los días trabajados.

**Ahora se revisará la columna "dob_years"**

In [41]:
#revisando la columna "dob_years"
bank_data["dob_years"].value_counts()

35    617
40    609
41    607
34    603
38    598
42    597
33    581
39    573
31    560
36    555
44    547
29    545
30    540
48    538
37    537
50    514
43    513
32    510
49    508
28    503
45    497
27    493
56    487
52    484
47    480
54    479
46    475
58    461
57    460
53    459
51    448
59    444
55    443
26    408
60    377
25    357
61    355
62    352
63    269
64    265
24    264
23    254
65    194
22    183
66    183
67    167
21    111
0     101
68     99
69     85
70     65
71     58
20     51
72     33
19     14
73      8
74      6
75      1
Name: dob_years, dtype: int64

El unico valor problemático es edad 0, se estudiará si es mejor reemplazarlo por la media o la mediana según género.

In [42]:
#revisando la media y mediana de edad por género
print(bank_data.groupby("gender")["dob_years"].median())
print(bank_data.groupby("gender")["dob_years"].mean())

gender
F      44.0
M      40.0
XNA    24.0
Name: dob_years, dtype: float64
gender
F      44.471972
M      40.993825
XNA    24.000000
Name: dob_years, dtype: float64


 La mediana y la media son parecidas para el género femenino y masculino, por separado, por lo que se decide utilizar la mediana de cada uno para renombrar las edades 0.

In [43]:
#calculando las medianas de cada género
female_median_age = bank_data[bank_data["gender"] == "F"]["dob_years"].median()
male_median_age = bank_data[bank_data["gender"] == "M"]["dob_years"].median()

#reemplazando los valores para cada género en la columna "dob_years"
bank_data.loc[(bank_data["gender"] == "F")&(bank_data["dob_years"] == 0), "dob_years"] = int(female_median_age)
bank_data.loc[(bank_data["gender"] == "M")&(bank_data["dob_years"] == 0), "dob_years"] = int(male_median_age)

In [44]:
#comprobando el resultado
bank_data["dob_years"].unique()

array([42, 36, 33, 32, 53, 27, 43, 50, 35, 41, 40, 65, 54, 56, 26, 48, 24,
       21, 57, 67, 28, 63, 62, 47, 34, 68, 25, 31, 30, 20, 49, 37, 45, 61,
       64, 44, 52, 46, 23, 38, 39, 51, 59, 29, 60, 55, 58, 71, 22, 73, 66,
       69, 19, 72, 70, 74, 75], dtype=int64)

In [45]:
#arreglando la columna de días trabajados que corresponden a más de 100 años para el valor "dob_years" = 0
bank_data.loc[bank_data["days_employed"] > 10000, "days_employed"] = (bank_data["dob_years"]-15)*365

In [46]:
#arreglando su valor de años trabajados
bank_data["years_employed"] = (bank_data["days_employed"])/365

In [47]:
#comprobando que se arreglaron los días trabajados que tenían valor mayor a 900 años
print(bank_data["days_employed"].max())
print(bank_data["years_employed"].max())

20805.0
57.0


**Ahora se revisará la columna "family_status"**

In [48]:
#viendo los valores de la columna "family_status"
bank_data["family_status"].unique()

array(['married', 'civil partnership', 'widow / widower', 'divorced',
       'unmarried'], dtype=object)

In [49]:
#quitando los espacios
bank_data["family_status"] = bank_data["family_status"].replace("widow / widower", "widow/widower")

In [50]:
#verificando los cambios
bank_data["family_status"].unique()

array(['married', 'civil partnership', 'widow/widower', 'divorced',
       'unmarried'], dtype=object)

**Ahora se revisará la columna "gender"**

In [51]:
#viendo los valores en la columna "gender"
bank_data["gender"].value_counts()

F      14236
M       7288
XNA        1
Name: gender, dtype: int64

Existe una persona sin género, quizás no quizo ingresarlo, se revisará que se puede hacer con el dato faltante.

In [52]:
#revisando la fila con género "XNA" para ver si hay información que ayude a decidir
bank_data[bank_data["gender"] == "XNA"]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,years_employed
10701,0,2358.600502,24,some college,2,civil partnership,1,XNA,business,0,32624.825,buy real estate,6.461919


El valor problemático es 1 entre 21525, por lo que lo que se decida con este dato afectará al mínimo los resultados. Además los créditos no deben depender del género de la persona, por lo que si reemplazamos el dato por "F" o "M" da exactamente lo mismo.

In [53]:
bank_data["gender"] = bank_data["gender"].replace("XNA", "F")

In [54]:
#comprobando que se reemplazó exitosamente
bank_data["gender"].unique()

array(['F', 'M'], dtype=object)

**Ahora se revisara la columna "income_type"**

In [55]:
#viendo los valores en la columna "income_type"
bank_data["income_type"].value_counts()

employee                       11119
business                        5085
retiree                         3856
civil servant                   1459
unemployed                         2
entrepreneur                       2
student                            1
paternity / maternity leave        1
Name: income_type, dtype: int64

In [56]:
#quitando los espacios
bank_data["income_type"] = bank_data["income_type"].replace("paternity / maternity leave", "paternity/maternity leave")

In [57]:
#verificando los cambios
bank_data["income_type"].value_counts()

employee                     11119
business                      5085
retiree                       3856
civil servant                 1459
unemployed                       2
entrepreneur                     2
student                          1
paternity/maternity leave        1
Name: income_type, dtype: int64

**Ahora veremos si tenemos datos duplicados.**

In [58]:
#comprobar los duplicados
duplicated_bank_data = bank_data[bank_data.duplicated()]
duplicated_bank_data

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,years_employed
2849,0,NaN,41,secondary education,1,married,0,F,employee,0,NaN,purchase of the house for my family,NaN
3290,0,NaN,58,secondary education,1,civil partnership,1,F,retiree,0,NaN,to have a wedding,NaN
4182,1,NaN,34,bachelor's degree,0,civil partnership,1,F,employee,0,NaN,wedding ceremony,NaN
4851,0,NaN,60,secondary education,1,civil partnership,1,F,retiree,0,NaN,wedding ceremony,NaN
5557,0,NaN,58,secondary education,1,civil partnership,1,F,retiree,0,NaN,to have a wedding,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
20702,0,NaN,64,secondary education,1,married,0,F,retiree,0,NaN,supplementary education,NaN
21032,0,NaN,60,secondary education,1,married,0,F,retiree,0,NaN,to become educated,NaN
21132,0,NaN,47,secondary education,1,married,0,F,employee,0,NaN,housing renovation,NaN
21281,1,NaN,30,bachelor's degree,0,married,0,F,employee,0,NaN,buy commercial real estate,NaN


In [59]:
#observando la columna donde hay datos faltantes
duplicated_bank_data["days_employed"].unique()

array([nan])

Parece ser que los datos duplicados son solo de las filas donde hay datos faltantes, y al tener datos faltantes, no nos entregan demasiada información, por lo que procederemos a eliminar los datos duplicados.

In [60]:
#eliminando las filas duplicadas
bank_data = bank_data.drop_duplicates().reset_index(drop = True)

In [61]:
#comprobando que se eliminaron los duplicados
bank_data[bank_data.duplicated()]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,years_employed


In [62]:
#comprobando el tamaño del DataFrame después de eliminar los duplicados
bank_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21454 entries, 0 to 21453
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21454 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21454 non-null  int64  
 3   education         21454 non-null  object 
 4   education_id      21454 non-null  int64  
 5   family_status     21454 non-null  object 
 6   family_status_id  21454 non-null  int64  
 7   gender            21454 non-null  object 
 8   income_type       21454 non-null  object 
 9   debt              21454 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21454 non-null  object 
 12  years_employed    19351 non-null  float64
dtypes: float64(3), int64(5), object(5)
memory usage: 2.1+ MB


El DataFrame original tenía 21525 filas, y el nuevo tiene 21454, ya que se eliminaron 71 filas que estaban duplicadas.

In [63]:
"La proporción entre el nuevo data frame y el original es: {:.2%}".format(21454/21525)

'La proporción entre el nuevo data frame y el original es: 99.67%'

In [64]:
"El nuevo DataFrame es un {:.2%} más pequeño que el original".format(1-(21454/21525))

'El nuevo DataFrame es un 0.33% más pequeño que el original'

# Trabajar con valores ausentes

### Restaurar valores ausentes en `total_income`

Recordemos que en las columnas "days_employed" y "total_income" tenemos valores ausentes.

In [65]:
print(bank_data[(bank_data["dob_years"] != float("nan"))]["dob_years"].min())
print(bank_data[(bank_data["dob_years"] != float("nan"))]["dob_years"].max())

19
75


In [66]:
#función que calcule la categoría de edad
def age_category(age):
    if age >= 19 and age < 29:
        return "19-28"
    elif age >= 29 and age < 39:
        return "29-38"
    elif age >= 39 and age < 49:
        return "39-48"
    elif age >= 49 and age < 59:
        return "49-58"
    elif age >= 59:
        return "59 o más"

In [67]:
#probando si la función funciona bien
print(age_category(20))
print(age_category(35))
print(age_category(48))
print(age_category(49))
print(age_category(80))

19-28
29-38
39-48
49-58
59 o más


In [68]:
#creando una nueva columna basada en la función
bank_data["age_category"] = bank_data["dob_years"].apply(age_category)

In [69]:
#comprobando que se creo la nueva columna en base a la función
bank_data.head(15)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,years_employed,age_category
0,1,8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house,23.116912,39-48
1,1,4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase,11.026860,29-38
2,0,5623.422610,33,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house,15.406637,29-38
3,3,4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education,11.300677,29-38
4,0,6368.633414,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding,17.448311,49-58
5,0,926.185831,27,bachelor's degree,0,civil partnership,1,M,business,0,40922.170,purchase of the house,2.537495,19-28
6,0,2879.202052,43,bachelor's degree,0,married,0,F,business,0,38484.156,housing transactions,7.888225,39-48
7,0,152.779569,50,secondary education,1,married,0,M,employee,0,21731.829,education,0.418574,49-58
8,2,6929.865299,35,bachelor's degree,0,civil partnership,1,F,employee,0,15337.093,having a wedding,18.985932,29-38
9,0,2188.756445,41,secondary education,1,married,0,M,employee,0,23108.150,purchase of the house for my family,5.996593,39-48


In [70]:
def get_nulls_by_group(df, nulls_col, groupby_cols):
  return df[nulls_col].isna().groupby(groupby_cols).agg(["mean", "sum", "count"]).reset_index()

In [71]:
get_nulls_by_group(bank_data, "total_income", bank_data["age_category"])

,age_category,mean,sum,count
0,19-28,0.093703,247,2636
1,29-38,0.097977,552,5634
2,39-48,0.096032,530,5519
3,49-58,0.105040,496,4722
4,59 o más,0.094461,278,2943


In [72]:
get_nulls_by_group(bank_data, "total_income", bank_data["income_type"])

,income_type,mean,sum,count
0,business,0.098661,501,5078
1,civil servant,0.099520,145,1457
2,employee,0.096536,1070,11084
3,entrepreneur,0.500000,1,2
4,paternity/maternity leave,0.000000,0,1
5,retiree,0.100810,386,3829
6,student,0.000000,0,1
7,unemployed,0.000000,0,2


In [73]:
get_nulls_by_group(bank_data, "total_income", bank_data["family_status"])

,family_status,mean,sum,count
0,civil partnership,0.100217,416,4151
1,divorced,0.093724,112,1195
2,married,0.096928,1196,12339
3,unmarried,0.101423,285,2810
4,widow/widower,0.098019,94,959


In [74]:
get_nulls_by_group(bank_data, "total_income", bank_data["education"])

,education,mean,sum,count
0,bachelor's degree,0.101714,534,5250
1,graduate degree,0.000000,0,6
2,primary education,0.074468,21,282
3,secondary education,0.097482,1479,15172
4,some college,0.092742,69,744


Debido a lo anterior, se puede concluir que los elementos ausentes son totalmente aleatorios, ya que básicamente en todas las categorías son el 10%, es decir, ninguna característica influye en que ese valor sea ausente.

In [75]:
#creando y mostrando tabla sin valores ausentes
bank_data_filtered = bank_data.dropna(subset = ["total_income"]).reset_index(drop = True)
bank_data_filtered

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,years_employed,age_category
0,1,8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house,23.116912,39-48
1,1,4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase,11.026860,29-38
2,0,5623.422610,33,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house,15.406637,29-38
3,3,4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education,11.300677,29-38
4,0,6368.633414,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding,17.448311,49-58
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19346,1,4529.316663,43,secondary education,1,civil partnership,1,F,business,0,35966.698,housing transactions,12.409087,39-48
19347,0,8640.278490,67,secondary education,1,married,0,F,retiree,0,24959.969,purchase of a car,23.671996,59 o más
19348,1,2113.346888,38,secondary education,1,civil partnership,1,M,employee,1,14347.610,property,5.789991,29-38
19349,3,3112.481705,38,secondary education,1,married,0,M,employee,1,39054.888,buying my own car,8.527347,29-38


In [76]:
#calculando la media del ingreso total
total_income_mean = bank_data_filtered["total_income"].mean()
total_income_mean

26787.56835465871

In [77]:
#calculando la mediana del ingreso total
total_income_median = bank_data_filtered["total_income"].median()
total_income_median

23202.87

In [78]:
bank_data_filtered.pivot_table(index="education", values="total_income", aggfunc="median")

,total_income
education,
bachelor's degree,28054.5310
graduate degree,25161.5835
primary education,18741.9760
secondary education,21836.5830
some college,25618.4640


In [79]:
bank_data_filtered.pivot_table(index="income_type", values="total_income", aggfunc="median")

,total_income
income_type,
business,27577.2720
civil servant,24071.6695
employee,22815.1035
entrepreneur,79866.1030
paternity/maternity leave,8612.6610
retiree,18962.3180
student,15712.2600
unemployed,21014.3605


In [80]:
pivot_total_income = bank_data_filtered.pivot_table(index=["income_type","education"], values="total_income", aggfunc="median")
print(pivot_total_income)

                                               total_income
income_type               education                        
business                  bachelor's degree      32285.6640
                          primary education      21887.8250
                          secondary education    25451.3100
                          some college           28778.7440
civil servant             bachelor's degree      27601.7775
                          graduate degree        17822.7570
                          primary education      23734.2870
                          secondary education    21864.4750
                          some college           25694.7750
employee                  bachelor's degree      26502.5190
                          graduate degree        31771.3210
                          primary education      20159.1860
                          secondary education    21848.8175
                          some college           24209.4300
entrepreneur              bachelor's deg

In [81]:
pivot_total_income.loc[("retiree","some college"),"total_income"]

19221.903

Para restaurar los valores ausentes de "total_income" se va a tomar en cuenta que los ingresos dependen del tipo de ingreso y del nivel educacional.

In [82]:
#función que se usará para crear columna sin datos ausentes de "total_income"
def filling_income(row):
    education = row["education"]
    income_type = row["income_type"]
    income = row["total_income"]
    if pd.isna(income):
        return pivot_total_income.loc[(income_type,education), "total_income"]
    return income

In [83]:
#comprobando si la función funciona correctamente
#la fila 12 tiene valor ausente
filling_income(bank_data.iloc[12])

18374.857

In [84]:
#comprobando que la función funcione correctamente
#la fila 41 tiene valor ausente
filling_income(bank_data.iloc[41])

21864.475

In [85]:
#comprobando que la función funcione correctamente
#la fila 5 no tiene valor ausente
filling_income(bank_data.iloc[5])

40922.17

In [86]:
#aplicando la función para rellenar los datos
bank_data["total_income"] = bank_data.apply(filling_income, axis = 1)

In [87]:
#verificando que no quedan datos ausentes en "total_income"
bank_data["total_income"].isna().sum()

0

In [88]:
#comprobando la información de la tabla
bank_data.head(15)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,years_employed,age_category
0,1,8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house,23.116912,39-48
1,1,4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase,11.026860,29-38
2,0,5623.422610,33,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house,15.406637,29-38
3,3,4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education,11.300677,29-38
4,0,6368.633414,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding,17.448311,49-58
5,0,926.185831,27,bachelor's degree,0,civil partnership,1,M,business,0,40922.170,purchase of the house,2.537495,19-28
6,0,2879.202052,43,bachelor's degree,0,married,0,F,business,0,38484.156,housing transactions,7.888225,39-48
7,0,152.779569,50,secondary education,1,married,0,M,employee,0,21731.829,education,0.418574,49-58
8,2,6929.865299,35,bachelor's degree,0,civil partnership,1,F,employee,0,15337.093,having a wedding,18.985932,29-38
9,0,2188.756445,41,secondary education,1,married,0,M,employee,0,23108.150,purchase of the house for my family,5.996593,39-48


Los valores ausentes en la columna "total_income" se han reemplazado exitosamente.

In [89]:
#comprobando el número de entradas en las columnas
bank_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21454 entries, 0 to 21453
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21454 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21454 non-null  int64  
 3   education         21454 non-null  object 
 4   education_id      21454 non-null  int64  
 5   family_status     21454 non-null  object 
 6   family_status_id  21454 non-null  int64  
 7   gender            21454 non-null  object 
 8   income_type       21454 non-null  object 
 9   debt              21454 non-null  int64  
 10  total_income      21454 non-null  float64
 11  purpose           21454 non-null  object 
 12  years_employed    19351 non-null  float64
 13  age_category      21454 non-null  object 
dtypes: float64(3), int64(5), object(6)
memory usage: 2.3+ MB


###  Restaurar valores en `days_employed`

Para restaurar los valores en "days_employed" hay que tener en cuenta la edad de las personas, y para calcular las medias y medianas de días trabajados según la edad, utilizaremos los rangos de edad.

In [90]:
#comprobando que los nulos se distribuyen igual que en "total_income"
get_nulls_by_group(bank_data, "days_employed", bank_data["age_category"])

,age_category,mean,sum,count
0,19-28,0.093703,247,2636
1,29-38,0.097977,552,5634
2,39-48,0.096032,530,5519
3,49-58,0.105040,496,4722
4,59 o más,0.094461,278,2943


La distribución de los valores nulos va a ser la misma que con "total_income", ya que faltaban ambos datos en las mismas filas.

In [91]:
#calculando la media de los días trabajados
days_employed_mean = bank_data_filtered["days_employed"].mean()
days_employed_mean

3287.261717934732

In [92]:
#calculando la mediana de los días trabajados
days_employed_median = bank_data_filtered["days_employed"].median()
days_employed_median

2192.155977727033

Se utilizará la mediana para rellenar los datos faltantes.

In [93]:
bank_data_filtered.pivot_table(index="income_type", values="days_employed", aggfunc="median")

,days_employed
income_type,
business,1547.382223
civil servant,2689.368353
employee,1574.202821
entrepreneur,520.848083
paternity/maternity leave,3296.759962
retiree,7504.455952
student,578.751554
unemployed,3934.727976


In [94]:
bank_data_filtered.pivot_table(index="age_category", values="days_employed", aggfunc="median")

,days_employed
age_category,
19-28,941.880042
29-38,1548.381992
39-48,2030.362978
49-58,3878.223510
59 o más,7828.976677


In [95]:
pivot_days_employed = bank_data_filtered.pivot_table(index=["age_category","income_type"], values="days_employed", aggfunc="median")
print(pivot_days_employed)

                                        days_employed
age_category income_type                             
19-28        business                      853.729307
             civil servant                1265.117765
             employee                      947.957386
             entrepreneur                  520.848083
             retiree                      2149.863988
             student                       578.751554
29-38        business                     1466.080981
             civil servant                2482.030008
             employee                     1481.945158
             retiree                      3772.467613
             unemployed                   2798.905438
39-48        business                     1862.259612
             civil servant                3551.609375
             employee                     1865.551891
             paternity/maternity leave    3296.759962
             retiree                      5232.810876
             unemployed     

Para restaurar los valores ausentes de "days_employed" se va a tomar en cuenta que los días trabajados dependen del tipo de ingreso y del rango de edad.

In [96]:
#función que se usará para rellenar los datos ausentes de "days_employed"
def filling_days_employed(row):
    age = row["age_category"]
    income_type = row["income_type"]
    days = row["days_employed"]
    if pd.isna(days):
        try:
            return pivot_days_employed.loc[(age,income_type), "days_employed"]
        except:
            return bank_data_filtered["days_employed"].median()
    return days

In [97]:
#comprobando si la función funciona correctamente
#la fila 12 tiene valor ausente
filling_days_employed(bank_data.iloc[12])

7991.237039387237

In [98]:
#aplicando la función para rellenar los datos
bank_data["days_employed"] = bank_data.apply(filling_days_employed, axis = 1)

In [99]:
#comprobando si funcionó la función
bank_data.head(15)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,years_employed,age_category
0,1,8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house,23.116912,39-48
1,1,4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase,11.026860,29-38
2,0,5623.422610,33,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house,15.406637,29-38
3,3,4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education,11.300677,29-38
4,0,6368.633414,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding,17.448311,49-58
5,0,926.185831,27,bachelor's degree,0,civil partnership,1,M,business,0,40922.170,purchase of the house,2.537495,19-28
6,0,2879.202052,43,bachelor's degree,0,married,0,F,business,0,38484.156,housing transactions,7.888225,39-48
7,0,152.779569,50,secondary education,1,married,0,M,employee,0,21731.829,education,0.418574,49-58
8,2,6929.865299,35,bachelor's degree,0,civil partnership,1,F,employee,0,15337.093,having a wedding,18.985932,29-38
9,0,2188.756445,41,secondary education,1,married,0,M,employee,0,23108.150,purchase of the house for my family,5.996593,39-48


In [100]:
#corrigiendo la columna "years_employed"
bank_data["years_employed"] = (bank_data["days_employed"])/365

In [101]:
# Comprueba las entradas en todas las columnas: asegúrate de que hayamos corregido to
bank_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21454 entries, 0 to 21453
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21454 non-null  int64  
 1   days_employed     21454 non-null  float64
 2   dob_years         21454 non-null  int64  
 3   education         21454 non-null  object 
 4   education_id      21454 non-null  int64  
 5   family_status     21454 non-null  object 
 6   family_status_id  21454 non-null  int64  
 7   gender            21454 non-null  object 
 8   income_type       21454 non-null  object 
 9   debt              21454 non-null  int64  
 10  total_income      21454 non-null  float64
 11  purpose           21454 non-null  object 
 12  years_employed    21454 non-null  float64
 13  age_category      21454 non-null  object 
dtypes: float64(3), int64(5), object(6)
memory usage: 2.3+ MB


Se logró rellenar todos los datos faltantes de la tabla.

## Clasificación de datos

In [102]:
#mostrando los valores de los datos seleccionados para la clasificación
bank_data[["children","family_status","total_income","purpose","debt"]]

,children,family_status,total_income,purpose,debt
0,1,married,40620.102,purchase of the house,0
1,1,married,17932.802,car purchase,0
2,0,married,23341.752,purchase of the house,0
3,3,married,42820.568,supplementary education,0
4,0,civil partnership,25378.572,to have a wedding,0
...,...,...,...,...,...
21449,1,civil partnership,35966.698,housing transactions,0
21450,0,married,24959.969,purchase of a car,0
21451,1,civil partnership,14347.610,property,1
21452,3,married,39054.888,buying my own car,1


In [103]:
#comprobando los valores únicos de la variable principal para la clasificacion
bank_data["debt"].unique()

array([0, 1], dtype=int64)

Se identifican dos grupos:
- 1 : si ha inclumplido alguna vez con el pago de un préstamo
- 0 : si nunca ha inclumplido con un pago

Observando las columnas que se van a estudiar, es claro que la columna "purpose" se debe categorizar, para un mejor estudio de ella.

In [104]:
#observando la columna "purpose" para crear categorías
bank_data["purpose"].unique()

array(['purchase of the house', 'car purchase', 'supplementary education',
       'to have a wedding', 'housing transactions', 'education',
       'having a wedding', 'purchase of the house for my family',
       'buy real estate', 'buy commercial real estate',
       'buy residential real estate', 'construction of own property',
       'property', 'building a property', 'buying a second-hand car',
       'buying my own car', 'transactions with commercial real estate',
       'building a real estate', 'housing',
       'transactions with my real estate', 'cars', 'to become educated',
       'second-hand car purchase', 'getting an education', 'car',
       'wedding ceremony', 'to get a supplementary education',
       'purchase of my own house', 'real estate transactions',
       'getting higher education', 'to own a car', 'purchase of a car',
       'profile education', 'university education',
       'buying property for renting out', 'to buy a car',
       'housing renovation', 'going

In [105]:
#función para clasificar los datos en función de temas comunes de "purpose"
def purpose_category(purpose):
    if ("house" in purpose) or ("housing" in purpose) or ("property" in purpose) or ("real estate" in purpose):
        return "house/real estate"
    elif ("car" in purpose) or ("cars" in purpose):
        return "car"
    elif ("education" in purpose) or ("educated" in purpose) or ("university" in purpose):
        return "education"
    elif ("wedding" in purpose):
        return "wedding"
    else:
        return "other"

In [106]:
#creando una columna con las categorías y cuenta los valores en ellas
bank_data["purpose_category"] = bank_data["purpose"].apply(purpose_category)
bank_data["purpose_category"].value_counts()

house/real estate    10811
car                   4306
education             4013
wedding               2324
Name: purpose_category, dtype: int64

También se le puede dar una clasificación a la columna "total_income".

In [107]:
#revisando los datos numéricos en la columna seleccionada para la clasificación
bank_data["total_income"].unique()

array([40620.102, 17932.802, 23341.752, ..., 14347.61 , 39054.888,
       13127.587])

In [108]:
#obteniendo estadísticas resumidas para la columna
print(bank_data["total_income"].min())
print(bank_data["total_income"].max())
print(bank_data["total_income"].mean())
print(bank_data["total_income"].median())

3306.762
362496.645
26472.56343644518
22993.273999999998


Se clasificará el ingreso de las personas en rangos de 10000 y luego habrá uno solo sobre los 100000 ya que son un porcentaje mucho menor.

In [109]:
#creando una función para clasificar en diferentes grupos numéricos basándose en rangos
def income_category(income):
    if income <= 10000:
        return "0-10000"
    elif (income > 10000) and (income <= 20000):
        return "10000-20000"
    elif (income > 20000) and (income <= 30000):
        return "20000-30000"
    elif (income > 30000) and (income <= 40000):
        return "30000-40000"
    elif (income > 40000) and (income <= 50000):
        return "40000-50000"
    elif (income > 50000) and (income <= 60000):
        return "50000-60000"
    elif (income > 60000) and (income <= 70000):
        return "60000-70000"
    elif (income > 70000) and (income <= 80000):
        return "70000-80000"
    elif (income > 80000) and (income <= 90000):
        return "80000-90000"
    elif (income > 90000) and (income <= 100000):
        return "90000-100000"
    else:
        return "100000-more"

In [110]:
#creando una función para clasificar en diferentes grupos numéricos basándose en rangos
def income_category2(income):
    if income <= 50000:
        return "0-50000"
    elif (income > 50000) and (income <= 100000):
        return "50000-100000"
    elif (income > 100000) and (income <= 150000):
        return "100000-150000"
    elif (income > 150000) and (income <= 200000):
        return "150000-200000"
    elif (income > 250000) and (income <= 300000):
        return "250000-300000"
    else:
        return "300000-more"

In [111]:
#creando una columna con categorías de ingreso
bank_data["income_category"] = bank_data["total_income"].apply(income_category)

In [112]:
#creando una segunda columna con otras categorías de ingreso
bank_data["income_category2"] = bank_data["total_income"].apply(income_category2)

In [113]:
#contando los valores de cada categoría para ver la distribución
bank_data["income_category"].value_counts()

20000-30000     7648
10000-20000     6774
30000-40000     3293
40000-50000     1492
0-10000          926
50000-60000      648
60000-70000      294
70000-80000      157
100000-more       99
80000-90000       83
90000-100000      40
Name: income_category, dtype: int64

In [114]:
#contando los valores de cada categoría para ver la distribución
bank_data["income_category2"].value_counts()

0-50000          20133
50000-100000      1222
100000-150000       71
150000-200000       17
300000-more          7
250000-300000        4
Name: income_category2, dtype: int64

## Comprobación de las hipótesis


**¿Existe una correlación entre tener hijos y pagar a tiempo?**

In [115]:
# Calcular la tasa de incumplimiento en función del número de hijos
bank_data.pivot_table(index='children', values='debt', aggfunc='mean')

,debt
children,
0,0.075438
1,0.091658
2,0.094925
3,0.081818
4,0.097561
5,0.000000


**Conclusión**

No se puede afirmar que el incumplimiento del pago de un préstamo dependa del número de hijos ya qye no existe una correlación entre ellos.

**¿Existe una correlación entre la situación familiar y el pago a tiempo?**

In [116]:
#calculando la tasa de incumplimiento en función del estado familiar
bank_data.pivot_table(index='family_status', values='debt', aggfunc='mean')

,debt
family_status,
civil partnership,0.093471
divorced,0.071130
married,0.075452
unmarried,0.097509
widow/widower,0.065693


**Conclusión**

Hay mayor tasa de incumplimiento cuando la persona está soltera, pero tampoco se logra observar una correlación entre el estado familiar y el incumplimiento de pago del préstamo.

**¿Existe una correlación entre el nivel de ingresos y el pago a tiempo?**

In [117]:
#calculando la tasa de incumplimiento basada en el nivel de ingresos
bank_data.pivot_table(index='income_category', values='debt', aggfunc='mean')

,debt
income_category,
0-10000,0.062635
10000-20000,0.085621
100000-more,0.060606
20000-30000,0.086166
30000-40000,0.075919
40000-50000,0.068365
50000-60000,0.083333
60000-70000,0.054422
70000-80000,0.050955


In [118]:
#calculando la tasa de incumplimiento basada en el nivel de ingresos
bank_data.pivot_table(index='income_category2', values='debt', aggfunc='mean')

,debt
income_category2,
0-50000,0.081905
100000-150000,0.056338
150000-200000,0.058824
250000-300000,0.000000
300000-more,0.142857
50000-100000,0.070376


**Conclusión**

Se hicieron 2 clasificaciones del ingreso, una de 10000 en 10000 y otra de 50000 en 50000 para ver la variación de los % de incumplimiento. En la que va de 10000 en 10000 no se puede observar con mucha claridad, más bien los % no varían mucho entre ellos. Por otro lado cuando varía en 50000, se puede observar que las personas con menos ingresos tienen mayor % de incumplimiento, salvo las que ganan sobre 300000, pero eso se puede deber a los pocos datos que hay en este rango.

**¿Cómo afecta el propósito del crédito a la tasa de incumplimiento?**

In [119]:
# Consulta los porcentajes de tasa de incumplimiento para cada propósito del crédito y analízalos
bank_data.pivot_table(index='purpose_category', values='debt', aggfunc='mean')

,debt
purpose_category,
car,0.093590
education,0.092200
house/real estate,0.072334
wedding,0.080034


**Conclusión**

La mayor tasa de incumplimiento se encuentra en las personas que piden el préstamo para el auto y la educación, pero no pareciera ser que hay una correlación entre el propósito y el incumplimiento.


# Conclusión general 

Se tuvo que hacer un trabajo profundo en la base de datos, ya que se registraban datos incongruentes y datos faltantes. Primero se arreglaron los datos incongruentes en las columnas "education", "children","family_status","days_employed", "dob_years" y "gender" para luego rellenar los datos faltantes en "total_income" y en "days_employed".

Todos los arreglos fueron fundamentados.

Además, se hizo un estudio para ver como afectan algunas de las variables en el incumplimiento del pago del préstamo. Para esto se estudio el ingreso total, la cantidad de hijos, el estatus familiar y el propósito del prestamo.

- La cantidad de hijos no presentó ninguna correlación, por lo que el incumplimiento del pago no depende de esta variable.

- Según el estatus familiar, los solteros son los que tienen mayor tasa de incumplimiento, junto con los que están en unión civil, y por otro lado, los divorciados y viudos son los con menos tasa. Esto no significa que tengan alguna correlación, pero si puede servir para estar pendiente al momento de entregar un préstamo.

- El ingreso total de las personas sin duda debe tener alguna correlación con el cumplimiento del préstamo, si bien los datos no muestran un resultado tan notorio, se puede observar que las personas con menor ingreso tienen más problemas para cumplir con el pago del préstamo.

- Por último, la tasa de incumplimiento según el propósito es más alta para comprar un auto y más baja para pagar una boda.

Para mejorar el sistema de puntuación de crédito se debería estudiar la correlación múltiple, la cual entregaría valores más exactos, pero por el momento se puede concluir que la variable que más tiene correlación con el incumplimiento de pago es el ingreso total de cada persona.